In [ ]:
 # 1. Top 5 cities with highest spends and their percentage contribution of total credit card spends

WITH CitySpends AS (
    SELECT
        city,
        SUM(amount) AS total_spends
    FROM
        YourTableName
    GROUP BY
        city
),
TotalSpends AS (
    SELECT SUM(total_spends) AS overall_total FROM CitySpends
)
SELECT TOP 5
    city,
    total_spends,
    (total_spends * 100.0 / (SELECT overall_total FROM TotalSpends)) AS percentage_contribution
FROM
    CitySpends
ORDER BY
    total_spends DESC;


In [ ]:
# 2. Highest spend month and amount spent in that month for each card type

SELECT
    card_type,
    FORMAT(CONVERT(DATE, date, 103), 'MMMM-yyyy') AS month_year,
    SUM(amount) AS total_spends
FROM
    YourTableName
GROUP BY
    card_type,
    FORMAT(CONVERT(DATE, date, 103), 'MMMM-yyyy')
ORDER BY
    card_type,
    total_spends DESC;


In [ ]:
# 3. Transaction details for each card type when it reaches a cumulative of 1,000,000 total spends

WITH CumulativeSpends AS (
    SELECT
        *,
        SUM(amount) OVER (PARTITION BY card_type ORDER BY date) AS cumulative_spends
    FROM
        YourTableName
)
SELECT *
FROM
    CumulativeSpends
WHERE
    cumulative_spends >= 1000000
    AND cumulative_spends - amount < 1000000;


In [ ]:
# 4. City with the lowest percentage spend for Gold card type

WITH GoldSpends AS (
    SELECT
        city,
        SUM(amount) AS city_gold_spends
    FROM
        YourTableName
    WHERE
        card_type = 'Gold'
    GROUP BY
        city
),
TotalGoldSpends AS (
    SELECT SUM(city_gold_spends) AS total_gold_spends FROM GoldSpends
)
SELECT
    city,
    (city_gold_spends * 100.0 / (SELECT total_gold_spends FROM TotalGoldSpends)) AS percentage_contribution
FROM
    GoldSpends
ORDER BY
    percentage_contribution ASC
FETCH FIRST 1 ROW ONLY;


In [ ]:
# 5. City, highest expense type, and lowest expense type

WITH ExpenseTypeSpends AS (
    SELECT
        city,
        exp_Type,
        SUM(amount) AS total_spends
    FROM
        YourTableName
    GROUP BY
        city, exp_Type
),
CityExpenses AS (
    SELECT
        city,
        exp_Type,
        total_spends,
        ROW_NUMBER() OVER (PARTITION BY city ORDER BY total_spends DESC) AS rank_high,
        ROW_NUMBER() OVER (PARTITION BY city ORDER BY total_spends ASC) AS rank_low
    FROM
        ExpenseTypeSpends
)
SELECT
    city,
    MAX(CASE WHEN rank_high = 1 THEN exp_Type END) AS highest_expense_type,
    MAX(CASE WHEN rank_low = 1 THEN exp_Type END) AS lowest_expense_type
FROM
    CityExpenses
GROUP BY
    city;





In [ ]:
# 6. Percentage contribution of spends by females for each expense type

WITH FemaleSpends AS (
    SELECT
        exp_Type,
        SUM(amount) AS female_spends
    FROM
        YourTableName
    WHERE
        gender = 'F'
    GROUP BY
        exp_Type
),
TotalSpends AS (
    SELECT
        exp_Type,
        SUM(amount) AS total_spends
    FROM
        YourTableName
    GROUP BY
        exp_Type
)
SELECT
    f.exp_Type,
    (female_spends * 100.0 / t.total_spends) AS percentage_contribution
FROM
    FemaleSpends f
JOIN
    TotalSpends t ON f.exp_Type = t.exp_Type;


In [ ]:
# 7. Card and expense type combination with highest month-over-month growth in January 2014

WITH MonthlySpends AS (
    SELECT
        card_type,
        exp_Type,
        FORMAT(CONVERT(DATE, date, 103), 'yyyy-MM') AS month_year,
        SUM(amount) AS monthly_spends
    FROM
        YourTableName
    GROUP BY
        card_type, exp_Type, FORMAT(CONVERT(DATE, date, 103), 'yyyy-MM')
),
MonthOverMonthGrowth AS (
    SELECT
        card_type,
        exp_Type,
        month_year,
        monthly_spends,
        LAG(monthly_spends) OVER (PARTITION BY card_type, exp_Type ORDER BY month_year) AS prev_month_spends,
        (monthly_spends - LAG(monthly_spends) OVER (PARTITION BY card_type, exp_Type ORDER BY month_year)) AS mom_growth
    FROM
        MonthlySpends
)
SELECT
    card_type,
    exp_Type,
    month_year,
    mom_growth
FROM
    MonthOverMonthGrowth
WHERE
    month_year = '2014-01'
ORDER BY
    mom_growth DESC
FETCH FIRST 1 ROW ONLY;


In [ ]:
# 8. City with highest total spend to total number of transactions ratio during weekends

WITH WeekendTransactions AS (
    SELECT
        city,
        SUM(amount) AS total_spends,
        COUNT(*) AS total_transactions
    FROM
        YourTableName
    WHERE
        DATEPART(dw, CONVERT(DATE, date, 103)) IN (1, 7) -- Assuming 1=Sunday, 7=Saturday in SQL Server's DATEPART
    GROUP BY
        city
)
SELECT
    city,
    (total_spends * 1.0 / total_transactions) AS spend_to_transaction_ratio
FROM
    WeekendTransactions
ORDER BY
    spend_to_transaction_ratio DESC
FETCH FIRST 1 ROW ONLY;


In [ ]:
# 9. City that took the least number of days to reach its 500th transaction after the first transaction in that city

WITH CityTransactions AS (
    SELECT
        city,
        date,
        ROW_NUMBER() OVER (PARTITION BY city ORDER BY date) AS txn_number
    FROM
        YourTableName
),
FirstTo500Days AS (
    SELECT
        city,
        MIN(CASE WHEN txn_number = 500 THEN DATEDIFF(DAY, MIN(date) OVER (PARTITION BY city), date) END) AS days_to_500
    FROM
        CityTransactions
    GROUP BY
        city
)
SELECT
    city,
    days_to_500
FROM
    FirstTo500Days
ORDER BY
    days_to_500 ASC
FETCH FIRST 1 ROW ONLY;
